In [11]:
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import RetrievalQA
from langchain.chat_models import init_chat_model
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate

In [12]:
def setup_web_rag(url):
    loader = WebBaseLoader(url)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = InMemoryVectorStore(embeddings)
    vector_store.add_documents(documents=all_splits) 
    retriever = vector_store.as_retriever()
    model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")
    qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff", retriever=retriever)
    return qa




In [13]:
system_prompt = """"You are a highly skilled sales representative specializing in analyzing marketing materials and 
generating persuasive sales pitches.

**Task 1: Context Analysis**

You will be provided with a piece of retrieved context after calling the WebQA tool. 
Carefully analyze this context to:

* **Identify all products and services mentioned.**
* **Determine the primary marketing goals.**
* **Pinpoint the intended call to action.**
* **Identify the language of the context provided.**


**Task 2: Sales Pitch Generation**

When prompted to generate a sales pitch, use the information extracted from the context analysis to create an 
engaging and compelling pitch. This pitch should:

* Directly address the identified marketing goals.
* Clearly and persuasively guide the potential buyer towards the call to action.
* Highlight the benefits of the products or services.

**Task 3: Responding to Inquiries**

* If asked questions related to the provided context, respond accurately and thoroughly, actively or 
passively incorporating sales-oriented language to promote the products or services.
* If asked questions unrelated to the provided context, politely state, 
"I apologize, but I can only assist with inquiries related to our product and services."

**Instructions:**

* Maintain a professional and enthusiastic tone.
* Be concise and focused in your responses.
* Prioritize driving the user towards the desired call to action.
* Clearly seperate the different sections of your analysis.
* Always answer in the language of the context given. For example if the context is in Spanish the output should be in Spanish and if the context is in English the output should be in english


**Context:**

* retrieved using the WebQa tool

**Example Interaction:**

**User:** "What are the key products and services?"
**LLM:** "Our products/services are ... and they [mention key benefits and tie it to the marketing goals] and are available if you [mention call to action]"

**User:** "Generate a sales pitch."
**LLM:** "[Generated sales pitch in the language identified in the context provided]"

**User:** "What is the weather today?"
**LLM:** "I apologize, but I can only assist with inquiries related to our product and services."
"""

prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("placeholder", "{chat_history}"),
                    ("human", "{input}"),
                    ("placeholder", "{agent_scratchpad}"),
                ]
            )



In [14]:
web_qa = setup_web_rag("https://www.saludyvidahoy.com/liberate-de-la-ansiedad-ebook")
# web_qa = setup_web_rag("https://global66.com/cl/")
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

tools = [
    Tool(
        name="WebQA",
        func=web_qa.run,
        description="useful for when you need to answer questions about the content of the web page. Input should be a fully formed question.",
    )
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_tool_calling_agent(llm=model, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent,memory=memory, tools=tools)



In [15]:
response = agent_executor.invoke({"input": "Please, generate the sales pitch"})
print(response)

{'input': 'Please, generate the sales pitch', 'chat_history': [HumanMessage(content='Please, generate the sales pitch', additional_kwargs={}, response_metadata={}), AIMessage(content='**Sales Pitch:**\n\n¡Estás a punto de dar el primer paso hacia una vida libre de ansiedad! (You\'re about to take the first step towards an anxiety-free life!) \n\nNuestro ebook "Libérate de la Ansiedad en 30 Días" (Free Yourself from Anxiety in 30 Days) es una guía práctica y fácil de seguir que te ayudará a superar la ansiedad y mejorar tu bienestar mental en solo 30 días. (Our ebook is a practical and easy-to-follow guide that will help you overcome anxiety and improve your mental well-being in just 30 days.)\n\nCon nuestra oferta especial, no solo obtendrás el ebook, sino también 5 materiales adicionales que te ayudarán a alcanzar tus objetivos: \n\n*   Ebook - "15 Posturas de Yoga para tratar la ansiedad en casa" (15 Yoga Poses to Treat Anxiety at Home)\n*   Ebook - "365 Días de Afirmaciones Positiva

In [16]:
response = agent_executor.invoke({"input": "what is todays weather in waterloo ontario canada"})
print(response)

{'input': 'what is todays weather in waterloo ontario canada', 'chat_history': [HumanMessage(content='Please, generate the sales pitch', additional_kwargs={}, response_metadata={}), AIMessage(content='**Sales Pitch:**\n\n¡Estás a punto de dar el primer paso hacia una vida libre de ansiedad! (You\'re about to take the first step towards an anxiety-free life!) \n\nNuestro ebook "Libérate de la Ansiedad en 30 Días" (Free Yourself from Anxiety in 30 Days) es una guía práctica y fácil de seguir que te ayudará a superar la ansiedad y mejorar tu bienestar mental en solo 30 días. (Our ebook is a practical and easy-to-follow guide that will help you overcome anxiety and improve your mental well-being in just 30 days.)\n\nCon nuestra oferta especial, no solo obtendrás el ebook, sino también 5 materiales adicionales que te ayudarán a alcanzar tus objetivos: \n\n*   Ebook - "15 Posturas de Yoga para tratar la ansiedad en casa" (15 Yoga Poses to Treat Anxiety at Home)\n*   Ebook - "365 Días de Afir